In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score  
import pickle
import pandas as pd

# Vorbereitung

Für das Clustering müssen die Wörter in einem String zusammengefasst werden.

In [2]:
file = open("data/top100Series.pkl", "rb")
kb = pickle.load(file)

def list_to_string(data):
    documents = []
    for i in data.index:
        documents.append(" ".join(data[i]))
    return documents

list_of_strings = list_to_string(kb)

list_of_strings

['incident staff notification email click select rule member assign message account send information configuration button use service template save log bc oracle section right edit ribbon enable refer documentation frame new deploy configure notify step action state create online communication appropriate drop site business like leave inbox detail add activate base include determine work change group version test hand item check menu initial specify event center position condition inform verify table submit administrator receive run box enter sure need non additional trigger correctly access close management report currently profile regard common answer option double product set interface display amp manager',
 'rule incident assign staff member section specific refer business information action oracle documentation use include bc service expression logical condition configure practice assistance set field single criterium online answer product feature request specify define incoming b

# Erstellen der Cluster

list_strings = Artikel für die ein Cluster gebildet werden sollen

cluster = Anzahl der Cluster, die gebildet werden sollen 

In [3]:

def create_clusters(list_strings, cluster):

    # Prepare the coloumn Resolution for clustering
    vectorizer = TfidfVectorizer(stop_words='english')
    resolution_vec = vectorizer.fit_transform(list_strings)
    #print(resolution_vec)
    #print(resolution_vec)
    # Amount of clusters
    cluster = 6

    # Defines the model for clustering 
    model = KMeans(n_clusters=cluster, init='k-means++', max_iter=1000, n_init=1)
    model.fit(resolution_vec)


    # all words (as index of this word) will be assigned in a cluster
    order_centroids = model.cluster_centers_.argsort()[:, ::-1]

    # List of all terms who are present in row 'Resolution'
    terms = vectorizer.get_feature_names()
    #print(terms)

    # Print all Clusters
    # !!!!! Speicherung in ein DataFrame, Dictonary etc. evtl sinnvoll
    dict = {}
    for i in range(cluster):
        temp = []
        for ind in order_centroids[i, :100]:
            temp.append(terms[ind])
        dict["Cluster %d" % i] = temp
    return dict, vectorizer, model, resolution_vec

cluster, vectorizer, model, resolution_vec = create_clusters(list_of_strings, 7)
cluster

{'Cluster 0': ['insight',
  'similar',
  'affect',
  'effectively',
  'begin',
  'filter',
  'contain',
  'report',
  'view',
  'datum',
  'follow',
  'search',
  'feature',
  'performance',
  'presentation',
  'present',
  'track',
  'utilize',
  'output',
  'offer',
  'topic',
  'produce',
  'unique',
  'additionally',
  'tabular',
  'cache',
  'wide',
  'benefit',
  'community',
  'basis',
  'catalog',
  'gain',
  'approach',
  'bring',
  'situation',
  'easily',
  'print',
  'consider',
  'correction',
  'url',
  'target',
  'answer_xref',
  'identical',
  'input',
  'intlink',
  'id',
  'keyword',
  'https',
  'customer',
  'people',
  'component',
  'thread',
  'forum',
  'form',
  'subscribe',
  'schedule',
  'range',
  'subscription',
  'folder',
  'understand',
  'graphical',
  'analytics',
  'insert',
  'current',
  'index',
  'return',
  'infrastructure',
  'infect',
  'internal',
  'extlink',
  'external',
  'easy',
  'edu',
  'efficient',
  'egw_ar_aggr_',
  'egw_ar_cons_'

# Zuweisung der Cluster 

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

#vectorizer = TfidfVectorizer()

cluster_results = []#{'Resolution': [], 'Cluster': []}

indizes = kb.index.tolist()
index = 0

# vectorize each cell in Resolution and make a prediction for a cluster
for i in kb:
    temp = ' '.join(i)
    
    vec = vectorizer.transform([temp])
    prediction = model.predict(vec)
    # saves the result in the dictonary cluster_results
    cluster_results.append([i,int(prediction)])
    index += 1

# Transform the dictonary in a dataframe
cluster_view = pd.DataFrame(cluster_results, index = kb.index.tolist(), columns=['Text', 'Cluster'])

# shows cluster_view
cluster_view

,Text,Cluster
1,"[incident, staff, notification, email, click, ...",4
9,"[rule, incident, assign, staff, member, sectio...",1
17,"[field, email, incident, address, response, co...",2
31,"[service, bc, oracle, support, requirement, in...",5
43,"[file, attachment, workspace, incident, answer...",1
48,"[status, incident, type, answer, custom, set, ...",3
53,"[answer, html, feature, add, code, include, co...",1
55,"[html, answer, source, tag, mode, service, ora...",4
58,"[page, home, file, set, php, default, configur...",1
68,"[report, information, service, bc, oracle, man...",0


# Prüfung, der zugordneten Incidents

In [5]:
def proof_prediction(quest, data, ldamodel):
    richtige_index = []
    right_cl = 0 #number of right matches
    wrong_cl = 0 #number of wrong matches

    indizes = data.index.tolist()
    ind = 0

    for i in quest.index: # i = index of quest
        quest_index = i

        # Preparing the question 
        #inc1 = nlp(quest[i])
        inc1 = ' '.join(quest[i])
        incident = ([inc1])

        # Prediction
        incidentvec = vectorizer.transform(incident)
        prediction1 = ldamodel.predict(incidentvec) # Number of cluster
    
        # Cluster of Resolution
        c1 = data.Cluster.loc[i]
    
        # Cluster kb == cluster incident: right else: wrong
        if c1 == prediction1[0]:
            right_cl += 1
            richtige_index.append(indizes[ind])      
        else: 
            wrong_cl += 1
            #print(indizes[ind])
        ind += 1
  
    print('Richtig: ' + str(right_cl))
    print('Falsch: ' + str(wrong_cl))
  
    quote = 0
    if wrong_cl != 0:
        quote = round(right_cl / index, 4)
        print('Quote: ' + str(round(right_cl / index, 4)) + '%')
    return richtige_index, quote


# ********************************************
#clusterres = pd.read_csv('cluster_results.csv')

file = open("data/clean_Question.pkl", "rb")
questions = pickle.load(file)
questions

richtig_index, quot = proof_prediction(questions, cluster_view, model)

print(richtig_index)
print(quot)

Richtig: 13
Falsch: 17
Quote: 0.4333%
[9, 43, 53, 58, 84, 104, 120, 128, 134, 135, 151, 159, 172]
0.4333


In [6]:
import tqdm

def proof_prediction1(quest, kb):

    k = range(1,20, 1) # Number of Clusters
    max_quote = 0
    max_k = 0
    csv_result = []
  
    # Model for k Cluster
    for n in k:
        index__df = []
        cluster_results = [] 
        km = KMeans(n_clusters=n, init='k-means++', max_iter=1000, n_init=1)
        km = km.fit(resolution_vec)

        # Assign Cluster in Resolution 
        for i in kb.index:
            temp = ' '.join(kb[i])
            vec = vectorizer.transform([temp])
            prediction = km.predict(vec)

            # saves the result in a list cluster_results
            index__df.append(i)
            cluster_results.append([
                              kb[i], 
                              int(prediction)
                              ])
    
        # Transform the cluster_results in a dataframe with original indizes
        cluster_view = pd.DataFrame(cluster_results, index = index__df, columns=['Text', 'Cluster'])
    
        # Returns a list of kb with same cluster and quote of right assignments
        richtige_indizes, quote = proof_prediction(questions, cluster_view, km)
    
        csv_result.append([k, quote])
        print("\n")
    
        # Identifies the hightes quote
        if quote > max_quote:
            max_quote = quote
            max_k = k
  

    ###
    #inde = cleankb_resolution.index.tolist()
    ###
    #df = pd.DataFrame(csv_result, columns=['k', 'quote'])
    #df.to_csv('csv_result.csv', index=False)
  
    return max_quote, max_k




# ***************************************************************************

file = open("data/clean_Resolution.pkl", "rb")
cleankb = pickle.load(file)

file = open("data/clean_Question.pkl", "rb")
questions = pickle.load(file)
questions

quot, cluster = proof_prediction1(questions, cleankb)


#print(quot)
#print('Opt K: ' + str(cluster))

Richtig: 30
Falsch: 0


Richtig: 30
Falsch: 0


Richtig: 30
Falsch: 0


Richtig: 30
Falsch: 0


Richtig: 30
Falsch: 0


Richtig: 30
Falsch: 0


Richtig: 30
Falsch: 0


Richtig: 30
Falsch: 0


Richtig: 30
Falsch: 0


Richtig: 30
Falsch: 0


Richtig: 30
Falsch: 0


Richtig: 30
Falsch: 0


Richtig: 30
Falsch: 0


Richtig: 30
Falsch: 0


Richtig: 30
Falsch: 0


Richtig: 30
Falsch: 0


Richtig: 30
Falsch: 0


Richtig: 30
Falsch: 0


Richtig: 30
Falsch: 0




In [7]:
def test_cluster(incident, quest):
    # Identify the index of the question
    index = 0
    index_cluster = []
    indizes = quest.index.tolist()
  
    for i in quest:
        if i == incident[0]:
            index_cluster.append(indizes[index])
            index += 1

      # Predict a cluster
    incidentvec = vectorizer.transform(incident)
    prediction1 = model.predict(incidentvec)
  
    # print results
    print('Incident: ', incident[0]) # String
    print('Cluster: ', prediction1) # Cluster of Incident
  

    if len(index_cluster) > 0:  # Cluster of his resolution if exist
        print('Cluster of Resolution: ', cluster_view.Cluster[index_cluster[0]])
    else:
        print("Not in Resolution")
    print('index_cluster: ', index_cluster) #index of the incident in kb

In [8]:
file = open("data/clean_Question.pkl", "rb")
questions = pickle.load(file)
questions

# ******************************************
test_cluster(["Can I receive an email when users have requested help and an incident is created?"], questions)

Incident:  Can I receive an email when users have requested help and an incident is created?
Cluster:  [1]
Not in Resolution
index_cluster:  []
